# Customer Segmentation Using KMeans (RFM Analysis)

A comprehensive customer analysis that applies K-means algorithms to customer data using three key 
features such as Frequency, Recency, and Monetary value (RFM) to group customers and understand their 
purchasing behavior based on these three key features. This helps the business conduct customer-centric 
marketing more effectively.

**About The Dataset**:

This Online Retail II data set contains all the transactions occurring for a UK-based and registered, non-store online retail between 01/12/2009 and 09/12/2011.The company mainly sells unique all-occasion gift-ware. Many customers of the company are wholesalers.

**Variable Information**:
- **InvoiceNo**: Invoice number. Nominal. A 6-digit integral number uniquely assigned to each transaction. If this code starts with the letter 'c', it indicates a cancellation
- **StockCode**: Product (item) code. Nominal. A 5-digit integral number uniquely assigned to each distinct product.
- **Description**: Product (item) name. Nominal.
- **Quantity**: The quantities of each product (item) per transaction. Numeric.
- **InvoiceDate**: Invice date and time. Numeric. The day and time when a transaction was generated.
- **UnitPrice**: Unit price. Numeric. Product price per unit in sterling (Â£).
- **CustomerID**: Customer number. Nominal. A 5-digit integral number uniquely assigned to each customer.
- **Country**: Country name. Nominal. The name of the country where a customer resides.

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler



#settings to make numbers easier to read on display
pd.set_option('display.float_format', lambda x: "%.4f" %x)

import warnings

warnings.filterwarnings('ignore')

%matplotlib inline

### Prepare Data

#### Import

In [2]:
def wrangle(filepath:str):
    # Read file into dataframe
    df = pd.read_excel(filepath, sheet_name=0)

    return df

#### Explore

In [3]:
df = wrangle("online_retail_II.xlsx")
print(df.shape)
print(df.info())

(525461, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      525461 non-null  object        
 1   StockCode    525461 non-null  object        
 2   Description  522533 non-null  object        
 3   Quantity     525461 non-null  int64         
 4   InvoiceDate  525461 non-null  datetime64[ns]
 5   Price        525461 non-null  float64       
 6   Customer ID  417534 non-null  float64       
 7   Country      525461 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 32.1+ MB
None


In [4]:
# Statistiscal information
df.describe()[['Quantity', 'Price']]

,Quantity,Price
count,525461.0000,525461.0000
mean,10.3377,4.6888
min,-9600.0000,-53594.3600
25%,1.0000,1.2500
50%,3.0000,2.1000
75%,10.0000,4.2100
max,19152.0000,25111.0900
std,107.4241,146.1269


In [5]:
# check for missing value
df.isna().sum()

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64

Things we need to deal wIth:
- Notice how Quantity and Price columns have negative value
- The id column has missing values

Looking at data where we have missing value

In [6]:
df[df["Customer ID"].isna()].head(10)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
263,489464,21733,85123a mixed,-96,2009-12-01 10:52:00,0.0000,NaN,United Kingdom
283,489463,71477,short,-240,2009-12-01 10:52:00,0.0000,NaN,United Kingdom
284,489467,85123A,21733 mixed,-192,2009-12-01 10:53:00,0.0000,NaN,United Kingdom
470,489521,21646,NaN,-50,2009-12-01 11:44:00,0.0000,NaN,United Kingdom
577,489525,85226C,BLUE PULL BACK RACING CAR,1,2009-12-01 11:49:00,0.5500,NaN,United Kingdom
578,489525,85227,SET/6 3D KIT CARDS FOR KIDS,1,2009-12-01 11:49:00,0.8500,NaN,United Kingdom
1055,489548,22271,FELTCRAFT DOLL ROSIE,1,2009-12-01 12:32:00,2.9500,NaN,United Kingdom
1056,489548,22254,FELT TOADSTOOL LARGE,12,2009-12-01 12:32:00,1.2500,NaN,United Kingdom
1057,489548,22273,FELTCRAFT DOLL MOLLY,3,2009-12-01 12:32:00,2.9500,NaN,United Kingdom
1058,489548,22195,LARGE HEART MEASURING SPOONS,1,2009-12-01 12:32:00,1.6500,NaN,United Kingdom


Notice how rows with null "Customer Id" has prices with negative values, 0, or a value slightly greater than 0. We are going to remove cutomer_id rows with null values in our wrangle function.

**Looking at the rows with negative values for the quantity column**

In [7]:
print("shape: ", df[df["Quantity"] < 0].shape)
df[df["Quantity"] < 0].head()

shape:  (12326, 8)


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
178,C489449,22087,PAPER BUNTING WHITE LACE,-12,2009-12-01 10:33:00,2.9500,16321.0000,Australia
179,C489449,85206A,CREAM FELT EASTER EGG BASKET,-6,2009-12-01 10:33:00,1.6500,16321.0000,Australia
180,C489449,21895,POTTING SHED SOW 'N' GROW SET,-4,2009-12-01 10:33:00,4.2500,16321.0000,Australia
181,C489449,21896,POTTING SHED TWINE,-6,2009-12-01 10:33:00,2.1000,16321.0000,Australia
182,C489449,22083,PAPER CHAIN KIT RETRO SPOT,-12,2009-12-01 10:33:00,2.9500,16321.0000,Australia


In [8]:
df[df["Quantity"] < 0]['Invoice'].str.replace("[0-9]", "", regex=True).value_counts()

Invoice
C    10205
Name: count, dtype: int64

In [9]:
# Using regex to find rows of Invoice that do not have 6 digits
df[df['Invoice'].str.match("^\\d{6}$") == False].head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
178,C489449,22087,PAPER BUNTING WHITE LACE,-12,2009-12-01 10:33:00,2.9500,16321.0000,Australia
179,C489449,85206A,CREAM FELT EASTER EGG BASKET,-6,2009-12-01 10:33:00,1.6500,16321.0000,Australia
180,C489449,21895,POTTING SHED SOW 'N' GROW SET,-4,2009-12-01 10:33:00,4.2500,16321.0000,Australia
181,C489449,21896,POTTING SHED TWINE,-6,2009-12-01 10:33:00,2.1000,16321.0000,Australia
182,C489449,22083,PAPER CHAIN KIT RETRO SPOT,-12,2009-12-01 10:33:00,2.9500,16321.0000,Australia


In [10]:
# Removing all numbers from the Invoice leaving us with letters and empty string
df['Invoice'] = df['Invoice'].astype(str)
df['Invoice'].str.replace("[0-9]", "", regex=True).value_counts()

Invoice
     515252
C     10206
A         3
Name: count, dtype: int64

In [11]:
# Checking how many of the Invoice id starts with A
df[df["Invoice"].str.startswith("A")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
179403,A506401,B,Adjust bad debt,1,2010-04-29 13:36:00,-53594.3600,NaN,United Kingdom
276274,A516228,B,Adjust bad debt,1,2010-07-19 11:24:00,-44031.7900,NaN,United Kingdom
403472,A528059,B,Adjust bad debt,1,2010-10-20 12:04:00,-38925.8700,NaN,United Kingdom


In [12]:
# casting StockCode column to string type
df['StockCode'] = df['StockCode'].astype('str')

# using regex to find where rows of the StockCode doesn't match 5 digit
# and also contains other numbers and letters that are repeated
(df[(df['StockCode'].str.match('^\\d{5}$') == False) & 
    (df['StockCode'].str.match('^\\d{5}[a-zA-z]+$') == False)]['StockCode'].unique()
)

array(['POST', 'D', 'DCGS0058', 'DCGS0068', 'DOT', 'M', 'DCGS0004',
       'DCGS0076', 'C2', 'BANK CHARGES', 'DCGS0003', 'TEST001',
       'gift_0001_80', 'DCGS0072', 'gift_0001_20', 'DCGS0044', 'TEST002',
       'gift_0001_10', 'gift_0001_50', 'DCGS0066N', 'gift_0001_30',
       'PADS', 'ADJUST', 'gift_0001_40', 'gift_0001_60', 'gift_0001_70',
       'gift_0001_90', 'DCGSSGIRL', 'DCGS0006', 'DCGS0016', 'DCGS0027',
       'DCGS0036', 'DCGS0039', 'DCGS0060', 'DCGS0056', 'DCGS0059', 'GIFT',
       'DCGSLBOY', 'm', 'DCGS0053', 'DCGS0062', 'DCGS0037', 'DCGSSBOY',
       'DCGSLGIRL', 'S', 'DCGS0069', 'DCGS0070', 'DCGS0075', 'B',
       'DCGS0041', 'ADJUST2', '47503J ', 'C3', 'SP1002', 'AMAZONFEE'],
      dtype=object)

From the above data exploration on rows with negative quantity values, it was observed that most of them contain invoice id that starts with C which indicates cancellation as described in the data description section. Invoices that starts with the letter "A" has price value has negative and missing Customer_Id. Stock code that do not have 5 digits would also be filtered out.

In [ ]:
# Creating a wrangle function
def wrangle(filepath:str):
    # Read file into dataframe
    df = pd.read_excel(filepath, sheet_name=0)

    # get only invoice number with 6-digits
    df['Invoice'] = df['Invoice'].astype('str')
    masked = df['Invoice'].str.match('^\\d{6}$')
    df = df[masked]

    # get stock code with five-digit or five-digits followed by letter(s)
    df['StockCode'] = df['StockCode'].astype('str')
    masked_stockcode = (
        (df['StockCode'].str.match("^\\d{5}$")) |
        (df['StockCode'].str.match("^\\d{5}[a-zA-Z]+$"))
    )

    # drop Customer_id with null values
    df.dropna(subset='Customer ID', inplace=True)

    return df